In [61]:
from scipy.spatial.transform import Rotation as R
import json
import numpy as np

def create_homogeneous_matrix(xyz, quaternion):
    rotation_matrix = R.from_quat(quaternion).as_matrix()
    T = np.eye(4) 
    T[:3, :3] = rotation_matrix
    T[:3, 3] = xyz 

    return T

def pose_inv(pose):
    R = pose[:3, :3]
    T = np.eye(4)
    T[:3, :3] = R.T
    T[:3, 3] = - R.T @ np.ascontiguousarray(pose[:3, 3])
    return T


K = np.load("handeye/intrinsics_d415.npy")
T_WC = np.load("handeye/T_WC_head.npy")
DIR = "scissor"

with open(f"data/{DIR}/demo_bottlenecks.json") as f:
    dbn = json.load(f)
    
T_GRIP_EEF = create_homogeneous_matrix([0, 0, 0.136], [0, 0, 0, 1])
demo_waypoints = np.vstack([dbn[key] for key in dbn.keys()])
demo_waypoints

array([[ 0.50418126,  0.09185436,  0.48143709, -0.99837866,  0.01344973,
        -0.05489271,  0.00677801],
       [ 0.49349562, -0.20686429,  0.41968363, -0.9836049 , -0.08057539,
         0.04566163,  0.15473856],
       [ 0.45431572,  0.05573993,  0.28884844,  0.99737963,  0.05747579,
         0.03869391,  0.02081296],
       [ 0.48795565, -0.09312451,  0.28185395, -0.99430351,  0.0169949 ,
         0.01044858,  0.10470206],
       [ 0.45431572,  0.14573993,  0.28884844,  0.99737963,  0.05747579,
         0.03869391,  0.02081296]])

In [62]:
point_2Ds = []

for i in range(len(demo_waypoints)):
    point3d = pose_inv(T_WC) @ T_GRIP_EEF @ create_homogeneous_matrix(demo_waypoints[i, :3], demo_waypoints[i, 3:])
    
    # Project the 3D point onto the image plane
    point_image_homogeneous = np.dot(K, point3d[:3, 3])
    
    # Normalize the coordinates to get the 2D image point
    point_2D = point_image_homogeneous[:2] / point_image_homogeneous[2]

    point_2Ds.append(point_2D)

point_2Ds

[array([1134.18602041,   63.35327591]),
 array([591.82082126, 156.4616502 ]),
 array([967.88171055, 318.41246776]),
 array([762.80761621, 349.90925078]),
 array([1093.95793212,  319.87692355])]

In [63]:
from PIL import Image, ImageDraw

head_rgb = Image.open(f"data/{DIR}/demo_head_rgb.png")

# Create a drawing context
draw = ImageDraw.Draw(head_rgb)

for point_2D in point_2Ds:
    x, y, r = point_2D[0], point_2D[1], 5
    # Draw the circle (ellipse) with bounding box [(x-r, y-r), (x+r, y+r)]
    draw.ellipse((x-r, y-r, x+r, y+r), outline="red", width=3)


# Save or show the image
head_rgb.show()  # To display the image